In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [25]:
# Carga los datos desde el archivo CSV
datos = pd.read_csv('datossaber11.csv')

todas = ['periodo','estu_tipodocumento','cole_area_ubicacion','cole_bilingue','cole_calendario',
         'cole_caracter','cole_genero','estu_depto_presentacion','estu_nacionalidad','fami_estratovivienda',
         'fami_tieneautomovil','fami_tienecomputador','fami_tieneinternet','fami_tienelavadora','desemp_ingles',
         'punt_ingles','punt_matematicas','punt_sociales_ciudadanas','punt_c_naturales','punt_lectura_critica','punt_global']

columnas_seleccionadas = ['estu_tipodocumento','cole_area_ubicacion','cole_bilingue','cole_calendario',
         'cole_caracter','cole_genero','estu_depto_presentacion','estu_nacionalidad','fami_estratovivienda',
         'fami_tieneautomovil','fami_tienecomputador','fami_tieneinternet','fami_tienelavadora','desemp_ingles','punt_global']

# Crear un nuevo DataFrame con las columnas seleccionadas
data = datos[columnas_seleccionadas]

# Visualiza los primeros registros y verifica la estructura de tus datos
print(data.head())

# Selecciona las características (X) y la variable objetivo (y)
X = data.drop('punt_global', axis=1)  # Excluye la variable objetivo
y = data['punt_global']

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normaliza las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


   estu_tipodocumento  cole_area_ubicacion  cole_bilingue  cole_calendario  \
0                   1                    0              0                0   
1                   1                    0              0                0   
2                   1                    0              0                0   
3                   1                    0              0                0   
4                   1                    0              0                0   

   cole_caracter  cole_genero  estu_depto_presentacion  estu_nacionalidad  \
0              0            0                        0                  1   
1              1            0                        0                  1   
2              2            1                        0                  1   
3              2            0                        0                  1   
4              2            0                        1                  1   

   fami_estratovivienda  fami_tieneautomovil  fami_tienecomputador  

In [26]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)  # Capa de salida sin activación para regresión
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train_scaled, y_train, epochs=10, batch_size=128, validation_split=0.2)


Epoch 1/10


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2360/2360 ━━━━━━━━━━━━━━━━━━━━ 3s 855us/step - loss: 14854.9492 - val_loss: 1002.3784
Epoch 2/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 770us/step - loss: 993.2113 - val_loss: 970.9858
Epoch 3/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 754us/step - loss: 979.1117 - val_loss: 956.2996
Epoch 4/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 764us/step - loss: 970.0084 - val_loss: 961.2253
Epoch 5/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 783us/step - loss: 966.2864 - val_loss: 967.8290
Epoch 6/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 747us/step - loss: 966.6666 - val_loss: 956.1870
Epoch 7/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 734us/step - loss: 969.1772 - val_loss: 953.6797
Epoch 8/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 742us/step - loss: 960.8616 - val_loss: 957.1742
Epoch 9/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 797us/step - loss: 962.7267 - val_loss: 956.4124
Epoch 10/10
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 2s 767us/step - loss: 965.0673 - val_loss: 950.4236


In [27]:
# Evalúa el modelo en el conjunto de prueba
loss = model.evaluate(X_test_scaled, y_test)
print(f'Loss en el conjunto de prueba: {loss}')

2950/2950 ━━━━━━━━━━━━━━━━━━━━ 1s 481us/step - loss: 958.1245
Loss en el conjunto de prueba: 955.9346923828125


In [7]:
model.save('modelo_saber11.h5')

In [13]:
# Crear un nuevo conjunto de datos con características específicas
nuevas_caracteristicas = pd.DataFrame({
    'cole_bilingue': [1],  # Ejemplo de característica bilingüe
    'cole_calendario': [0],  # Ejemplo de calendario del colegio
    'fami_tienecomputador': [1],  # Ejemplo de tener computador
    # Otras características necesarias para la predicción
    # Asegúrate de incluir todas las características necesarias y en el mismo orden que se usaron durante el entrenamiento del modelo
})

# Normalizar las nuevas características utilizando el scaler ajustado a X_train_scaled
nuevas_caracteristicas_scaled = scaler.transform(nuevas_caracteristicas)

# Obtener la predicción del modelo para las nuevas características
prediccion_nuevas_caracteristicas = modelo_cargado.predict(nuevas_caracteristicas_scaled)

# Mostrar la predicción
print(f'La predicción del puntaje en Saber 11 para las nuevas características es: {prediccion_nuevas_caracteristicas[0][0]}')


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- cole_area_ubicacion
- cole_caracter
- cole_genero
- desemp_ingles
- estu_depto_presentacion
- ...
